In [4]:
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier
pd.options.display.float_format = '{:.4f}'.format

C:\Users\Martin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Martin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
training='training'
label='interest_integer'
clf = RandomForestClassifier(max_depth=5,criterion='gini', random_state=0,n_estimators=150,n_jobs=-1)
df,columns,good_encoders,dp_table,df_tokenizer=load_object(clf.__class__.__name__+'_thesis')
dp_table

,columns,best value,encoders,time
after numeric feature selection,"[price, len_features, len_photos, dayofweek, h...",0.6785,{},00:04:18.278668
after TOP N features,"[price, len_features, len_photos, dayofweek, h...",0.6760,{},00:07:30.255568
after numeric features aggregated by categorical features,"[price, len_features, len_photos, dayofweek, h...",0.6615,{},00:23:48.994151
after numeric features interactions,"[price, len_features, len_photos, dayofweek, h...",0.6523,{},00:28:54.161800
after categorical feature encoding,"[price, len_features, len_photos, dayofweek, h...",0.6165,{'building_id': <function CB_target_encoding a...,00:40:17.206325
after text feature tokenization plus PCA,"[price, len_features, len_photos, dayofweek, h...",0.6152,{'building_id': <function CB_target_encoding a...,01:14:41.866579


In [ ]:
df= pd.read_csv('ensemble_predictions')
clf = RandomForestClassifier(max_depth=5,criterion='entropy', random_state=0,n_estimators=150,n_jobs=-1)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)

In [9]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler


def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params ={
            'n_estimators':trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 2, 15),
            'criterion': trial.suggest_categorical('criterion', ['gini','entropy']),
            'max_features': trial.suggest_categorical('max_features', ['sqrt','log2']),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 200),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 200),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10,256) }
    
    print(params)
    clf=RandomForestClassifier(random_state=0,n_jobs=-1,**params)                                                  
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 272, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_split': 197, 'min_samples_leaf': 104, 'max_leaf_nodes': 19}
[I 2020-05-30 16:14:21,972] Finished trial#0 with value: 0.6241426873254026 with parameters: {'n_estimators': 272, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_split': 197, 'min_samples_leaf': 104, 'max_leaf_nodes': 19}. Best is trial#0 with value: 0.6241426873254026.
{'n_estimators': 311, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_split': 89, 'min_samples_leaf': 71, 'max_leaf_nodes': 226}
[I 2020-05-30 16:14:54,869] Finished trial#1 with value: 0.5966664477452294 with parameters: {'n_estimators': 311, 'max_depth': 7, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_split': 89, 'min_samples_leaf': 71, 'max_leaf_nodes': 226}. Best is trial#1 with value: 0.5966664477452294.
{'n_estimators': 188, 'max_depth': 14, 'criterion': 'gini', 'max_features': 'log2', 'min_sampl

In [12]:
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results.csv')

In [29]:
table

,n_estimators,max_depth,criterion,max_features,min_samples_split,min_samples_leaf,model_id,score
0,1059,10,entropy,log2,69,10,RandomForestClassifier_83d2a0_0.58,0.5797
1,777,28,gini,log2,72,89,RandomForestClassifier_22e392_0.588,0.5876
2,1100,22,gini,log2,41,88,RandomForestClassifier_e58453_0.587,0.5874
3,674,27,entropy,log2,79,73,RandomForestClassifier_d2cdc7_0.583,0.5830
4,615,26,entropy,log2,81,48,RandomForestClassifier_e4682e_0.577,0.5774
...,...,...,...,...,...,...,...,...
95,799,16,entropy,sqrt,7,10,RandomForestClassifier_03525f_0.563,0.5631
96,850,18,entropy,sqrt,10,13,RandomForestClassifier_195004_0.564,0.5637
97,775,19,entropy,sqrt,22,6,RandomForestClassifier_6ff8f3_0.562,0.5617
98,817,17,entropy,sqrt,12,1,RandomForestClassifier_a3630e_0.562,0.5617


## Best_grid_params

In [ ]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler


def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params ={
            'n_estimators':trial.suggest_int('n_estimators', 500, 1100),
            'max_depth': trial.suggest_int('max_depth', 10, 30),
            'criterion': trial.suggest_categorical('criterion', ['gini','entropy']),
            'max_features': trial.suggest_categorical('max_features', ['sqrt','log2']),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
           # 'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 400,1500) 
    }
    
    print(params)
    clf=RandomForestClassifier(random_state=0,n_jobs=-1,**params)                                                  
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results_extra.csv')

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 1059, 'max_depth': 10, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_split': 69, 'min_samples_leaf': 10}
[I 2020-06-01 15:42:54,628] Finished trial#0 with value: 0.5797116613462956 with parameters: {'n_estimators': 1059, 'max_depth': 10, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_split': 69, 'min_samples_leaf': 10}. Best is trial#0 with value: 0.5797116613462956.
{'n_estimators': 777, 'max_depth': 28, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_split': 72, 'min_samples_leaf': 89}
[I 2020-06-01 15:43:59,657] Finished trial#1 with value: 0.5876170320595234 with parameters: {'n_estimators': 777, 'max_depth': 28, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_split': 72, 'min_samples_leaf': 89}. Best is trial#0 with value: 0.5797116613462956.
{'n_estimators': 1100, 'max_depth': 22, 'criterion': 'gini', 'max_features': 'log2', 'min_samples_split': 41, 'min_samples_leaf': 88}
[I 2020-06-01 15:45:28,624] Finished trial

In [30]:
table.sort_values('score')

,n_estimators,max_depth,criterion,max_features,min_samples_split,min_samples_leaf,model_id,score
52,891,19,entropy,sqrt,11,4,RandomForestClassifier_a74ab1_0.561,0.5608
68,833,19,entropy,sqrt,11,4,RandomForestClassifier_69df4b_0.561,0.5608
93,805,19,entropy,sqrt,12,3,RandomForestClassifier_f67fbf_0.561,0.5612
62,840,20,entropy,sqrt,18,4,RandomForestClassifier_0cdc5a_0.561,0.5612
45,795,19,entropy,sqrt,17,5,RandomForestClassifier_2dccdf_0.561,0.5613
...,...,...,...,...,...,...,...,...
3,674,27,entropy,log2,79,73,RandomForestClassifier_d2cdc7_0.583,0.5830
27,914,16,entropy,sqrt,20,100,RandomForestClassifier_3aa3b2_0.584,0.5841
7,1044,20,gini,log2,37,76,RandomForestClassifier_bba429_0.585,0.5854
2,1100,22,gini,log2,41,88,RandomForestClassifier_e58453_0.587,0.5874


In [22]:
table.sort_values('score')

,n_estimators,max_depth,criterion,max_features,min_samples_split,min_samples_leaf,model_id,score
12,886,26,entropy,sqrt,20,3,RandomForestClassifier_37902a_0.561,0.5610
18,934,24,entropy,sqrt,20,3,RandomForestClassifier_9ad8c9_0.561,0.5612
11,875,25,entropy,sqrt,20,3,RandomForestClassifier_e6357b_0.561,0.5613
8,838,18,entropy,sqrt,19,4,RandomForestClassifier_7481b9_0.561,0.5614
6,765,19,entropy,sqrt,18,4,RandomForestClassifier_a3fc34_0.561,0.5614
13,927,26,entropy,sqrt,12,2,RandomForestClassifier_13c10f_0.562,0.5615
2,711,20,entropy,sqrt,20,5,RandomForestClassifier_617c05_0.562,0.5615
16,888,28,entropy,sqrt,5,6,RandomForestClassifier_70ed4a_0.562,0.5617
9,785,18,entropy,sqrt,16,8,RandomForestClassifier_6889de_0.562,0.5617
19,947,23,entropy,sqrt,18,5,RandomForestClassifier_e32fef_0.562,0.5617


In [20]:
table.sort_values('score')

,n_estimators,max_depth,criterion,max_features,min_samples_split,min_samples_leaf,model_id,score
12,598,19,entropy,sqrt,7,6,RandomForestClassifier_ea826c_0.561,0.5615
1,559,19,entropy,sqrt,20,5,RandomForestClassifier_5f3887_0.562,0.5616
11,600,20,entropy,sqrt,2,6,RandomForestClassifier_fa8a9a_0.562,0.5616
10,594,20,entropy,sqrt,4,6,RandomForestClassifier_e2a896_0.562,0.5616
14,553,15,entropy,sqrt,9,5,RandomForestClassifier_95d453_0.562,0.5618
19,430,36,entropy,sqrt,20,2,RandomForestClassifier_a3d525_0.562,0.5618
13,600,38,entropy,sqrt,8,7,RandomForestClassifier_8d15d3_0.562,0.5620
7,538,45,entropy,sqrt,19,4,RandomForestClassifier_84a6fd_0.562,0.5620
4,537,47,entropy,sqrt,15,9,RandomForestClassifier_e3f2f9_0.562,0.5621
17,560,25,entropy,sqrt,6,5,RandomForestClassifier_45c732_0.562,0.5623


In [15]:
table.sort_values('score')

,n_estimators,max_depth,criterion,max_features,min_samples_split,min_samples_leaf,max_leaf_nodes,model_id,score
15,360,22,entropy,sqrt,50,1,399,RandomForestClassifier_d27966_0.569,0.5691
16,354,21,entropy,sqrt,57,1,390,RandomForestClassifier_20fd58_0.569,0.5694
17,366,21,entropy,sqrt,58,13,398,RandomForestClassifier_9f5575_0.57,0.5695
14,302,21,entropy,sqrt,49,2,384,RandomForestClassifier_5340c7_0.57,0.5697
13,315,23,entropy,sqrt,8,4,394,RandomForestClassifier_41b0f4_0.57,0.5699
19,402,22,entropy,sqrt,70,12,367,RandomForestClassifier_3225c5_0.57,0.5703
12,301,25,entropy,sqrt,8,4,360,RandomForestClassifier_71cd06_0.571,0.5708
11,309,24,entropy,sqrt,8,5,345,RandomForestClassifier_286059_0.571,0.5712
10,340,23,entropy,sqrt,7,5,344,RandomForestClassifier_8f016b_0.571,0.5712
18,362,21,entropy,sqrt,28,1,320,RandomForestClassifier_bf5ce0_0.572,0.5717


In [ ]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params ={
            'n_estimators':trial.suggest_int('n_estimators', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 2, 15),
            'max_features': trial.suggest_categorical('max_features', ['sqrt','log2']),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 200),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 200) }
    
    print(params)
    clf=RandomForestClassifier(random_state=0,criterion='entropy',n_jobs=-1,**params)                                                  
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,5))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_RF_Ensemble.csv')

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'n_estimators': 694, 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 69, 'min_samples_leaf': 196}


In [26]:
table
table.to_csv('to_csv_grid_RF_Ensemble1.csv')

In [5]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

df= pd.read_csv('ensemble_predictions_v1.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params ={
            'n_estimators':trial.suggest_int('n_estimators', 800, 1200),
            'max_depth': trial.suggest_int('max_depth', 10, 20),
            'max_features': trial.suggest_categorical('max_features', ['log2']) }
    
    print(params)
    clf=RandomForestClassifier(random_state=0,criterion='entropy',n_jobs=-1,**params)                                                  
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,5))
    trial.set_user_attr('model_id',model_id)
    #save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

#table.to_csv(clf.__class__.__name__+'_grid_RF_Ensemble.csv')

C:\Users\Martin\anaconda3\lib\site-packages\optuna\_experimental.py:61: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  ExperimentalWarning,


{'n_estimators': 972, 'max_depth': 15, 'max_features': 'log2'}
[I 2020-06-07 21:30:07,980] Finished trial#0 with value: 0.5308399860096841 with parameters: {'n_estimators': 972, 'max_depth': 15, 'max_features': 'log2'}. Best is trial#0 with value: 0.5308399860096841.
{'n_estimators': 992, 'max_depth': 13, 'max_features': 'log2'}
[I 2020-06-07 21:38:16,199] Finished trial#1 with value: 0.5300692091325659 with parameters: {'n_estimators': 992, 'max_depth': 13, 'max_features': 'log2'}. Best is trial#1 with value: 0.5300692091325659.
{'n_estimators': 1051, 'max_depth': 13, 'max_features': 'log2'}



KeyboardInterrupt: 

In [ ]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params ={
            'n_estimators':trial.suggest_int('n_estimators', 800, 1200),
            'max_depth': trial.suggest_int('max_depth', 10, 20),
            'max_features': trial.suggest_categorical('max_features', ['log2']) }
    
    print(params)
    clf=RandomForestClassifier(random_state=0,criterion='entropy',n_jobs=-1,**params)                                                  
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,5))
    trial.set_user_attr('model_id',model_id)
    #save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss
n=100

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

#table.to_csv(clf.__class__.__name__+'_grid_RF_Ensemble.csv')

C:\Users\Martin\anaconda3\lib\site-packages\optuna\_experimental.py:61: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  ExperimentalWarning,


{'n_estimators': 972, 'max_depth': 15, 'max_features': 'log2'}
[I 2020-06-07 21:53:43,297] Finished trial#0 with value: 0.528330426411794 with parameters: {'n_estimators': 972, 'max_depth': 15, 'max_features': 'log2'}. Best is trial#0 with value: 0.528330426411794.
{'n_estimators': 992, 'max_depth': 13, 'max_features': 'log2'}
[I 2020-06-07 22:03:43,405] Finished trial#1 with value: 0.5274533011271773 with parameters: {'n_estimators': 992, 'max_depth': 13, 'max_features': 'log2'}. Best is trial#1 with value: 0.5274533011271773.
{'n_estimators': 1051, 'max_depth': 13, 'max_features': 'log2'}
[I 2020-06-07 22:14:21,680] Finished trial#2 with value: 0.5273759840284957 with parameters: {'n_estimators': 1051, 'max_depth': 13, 'max_features': 'log2'}. Best is trial#2 with value: 0.5273759840284957.
{'n_estimators': 1159, 'max_depth': 19, 'max_features': 'log2'}
[I 2020-06-07 22:28:16,336] Finished trial#3 with value: 0.5279330564422047 with parameters: {'n_estimators': 1159, 'max_depth': 19,